In [32]:
import os
import logging
from functools import partial

import numpy as np
import pandas as pd
import requests
import csv
import pathlib


config ={
'osmo':
{'username' : 'KingstonSmith', 
'password' : '', 
'postman_token' : '',
'osmo_url' : 'https://kingstonsmith.osmodata.com/restful/api/v4/'}}

def get_osmo_headers():
    return {
        "username": config["osmo"]["username"],
        "password": config["osmo"]["password"],
        "Cache-Control": "no-cache",
        "Postman-Token": config["osmo"]["postman_token"],
    }

def osmo_request(request_url, format="json"):
    headers = get_osmo_headers()
    url = f"https://kingstonsmith.osmodata.com/restful/api/v4/{request_url}"
    response = requests.get(url=url, headers=headers)
    if response.status_code != 200:
        message = "OSMO request failed with status %s"
        data = {"request_url": request_url}
        log.error(message, response.status_code, extra={"data": data})
        raise RuntimeError((message % response.status_code) + "\n" + str(data))

    if format == "json":
        return response.json()
    elif format == "dataframe":
        return pd.DataFrame(response.json())
    else:
        raise ValueError(f"Unknown format specified: {format}")
        
def getnominallines_all(client_ref):
    #returns list of dictionaries
    results = osmo_request(f"managementaccounts/getnominallines/{client_ref}")
    return results["records"]


def getnominalcategories(client_ref):
    #returns list of dictionaries
    results = osmo_request(f"managementaccounts/getnominalcategories/{client_ref}")
    return results["records"]

def getnominaljoins(client_ref):
    #returns list of dictionaries
    results = osmo_request(f"managementaccounts/getnominaljoins/{client_ref}")
    return results["records"]


def getfinancialyears(client_ref):
    #returns list of years
    results = osmo_request(f"managementaccounts/getavailablefinancialyears/{client_ref}")
    return [record["FinancialYear"] for record in results["records"]]

def getfinancialperiods(client_ref):
    #returns list of dictionaries
    results = osmo_request(f"managementaccounts/getavailablefinancialperiods/{client_ref}/0")
    return results["records"]

def gettrialbalancereport(client_ref, year, period):
    #returns list of dictionaries
    results = osmo_request(f"managementaccounts/gettrialbalancereport/{client_ref}/{year}/{period}/0/True")
    return results["records"]

def getnominalheaders(client_ref):
    #returns list of dictionaries
    results = osmo_request(f"managementaccounts/getnominalheaders/{client_ref}")
    return results["records"]


In [12]:
def get_client_codes(client_ref: str) -> pd.DataFrame:
    """
    Outer Joins nominalheaders and nominalcategories to get clientcodes.

    Returns:
        | index | NominalName | UniqueNominalCode | UserNominalCode | CategoryName | SubCategoryName |
    """
    # Get account name from unique nominal code
    nominal_headers = pd.DataFrame(getnominalheaders(client_ref))

    # Get the categories that Osmo applies to the client's nominal codes
    nominal_categories = pd.DataFrame(getnominalcategories(client_ref))

    # TODO for client z36nnj, no nominal categories are returned. requested from Osmo if this is a bug.
    # TODO if there are certain types of clients that don't get this info, we need to adjust the code

    # Joined outer to see if there are any instances of UniqueNominalCodes but not another in one
    df = nominal_headers.merge(nominal_categories, on="UniqueNominalCode", how="outer")
    #assert not df.isnull().values.any(), "This indicates there is missing data between headers + catagories"

    df = df.drop_duplicates(subset=["UniqueNominalCode"], keep="last").reset_index()

    return df

In [8]:
client_ref = 'z783e8'
years = pd.DataFrame(getfinancialperiods(client_ref))

years

# No 0, 1000 periods

,FinancialYear,FinancialYearStartMonth,MinimumPeriodAvailable,MaximumPeriodAvailable
0,2015,12,1,13
1,2016,12,1,13
2,2017,12,1,13
3,2018,12,1,13
4,2019,12,1,13
5,2020,12,1,1


In [5]:
tb = pd.DataFrame(gettrialbalancereport(client_ref, 2016, 1))


tb.query("NominalName == 'Creditors Control Account'")

# Gives -2,558.33 should be the current balance

,CategoryName,SubcategoryName,UniqueNominalCode,NominalName,PeriodBalance,YearToDateBalance
75,Liability,Accounts Payable,255,Creditors Control Account,23503.1,-2558.33


In [13]:
df = pd.DataFrame(getnominallines_all(client_ref))
codes = get_client_codes(client_ref)

df = df.sort_values(['UpdateRef', 'CreateRef'], ascending=[False, False])
df.drop_duplicates(subset=['UniqueId'], keep='first', inplace=True)

q = df.query("NominalYear <= 2016").query("not (NominalYear == 2016 and NominalPeriod > 1)")

X = q.merge(codes, how='left', on='UniqueNominalCode')

pd.DataFrame(X.groupby(["UniqueNominalCode","NominalName"])["ValueB"].sum().round(3)).query("NominalName == 'Creditors Control Account'")

# Gives -2576.55 difference of 18.22 to the trial balance.

,,ValueB
UniqueNominalCode,NominalName,
255,Creditors Control Account,-2576.55


In [17]:
df.query("UniqueNominalCode == '255' and BalanceTransaction == 'Y'")

# no OSMO journals within dataset for Creditors Control Account (Even when not droping duplicates)

,AgreementRef,CreateRef,UpdateRef,UniqueId,JournalNo,JournalType,JournalSource,ValueB,UniqueNominalCode,UserNominalCode,...,NominalYear,NominalPeriod,BalanceTransaction,TimeStampModified,UserNameModified,TimeStampCreated,UserNameCreated,Posted,DeletedAtSource,NominalJoins


In [31]:
periods = [str(x) for x in range(1,13)]

df.query(f"UniqueNominalCode == '255' and NominalPeriod not in {periods}")

#No transactions no within period 1 to 12.

,AgreementRef,CreateRef,UpdateRef,UniqueId,JournalNo,JournalType,JournalSource,ValueB,UniqueNominalCode,UserNominalCode,...,NominalYear,NominalPeriod,BalanceTransaction,TimeStampModified,UserNameModified,TimeStampCreated,UserNameCreated,Posted,DeletedAtSource,NominalJoins
